In [62]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src.dbc import TimeDivisionKMeans
from src.dbc.utils import data_preprocessing, dimension_reduction, calc_contribution

In [66]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
df = data_preprocessing(xlsx)

# 3. dimension_reduction
df = dimension_reduction(df)

# 4. day clustering
copy_df = df.copy()

In [67]:
# 4. day clustering
df = copy_df[:24]
size = 3
tdKMeans = TimeDivisionKMeans(df, size=size)

df.head()

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
2018-05-01 00:00:00,0.341,0.121,0.245,0.009,0.085,0.154,0.161,0.311,0.358,0.177,...,0.372,0.181,0.322,0.374,0.571,0.453,0.161,0.127,0.259,0.364
2018-05-01 01:00:00,0.337,0.186,0.285,0.010,0.071,0.127,0.179,0.225,0.206,0.178,...,0.245,0.238,0.292,0.382,0.327,0.352,0.172,0.133,0.234,0.286
2018-05-01 02:00:00,0.324,0.214,0.487,0.009,0.061,0.139,0.182,0.221,0.150,0.152,...,0.245,0.250,0.287,0.320,0.359,0.366,0.242,0.219,0.197,0.368
2018-05-01 03:00:00,0.319,0.250,0.300,0.009,0.067,0.132,0.172,0.274,0.142,0.205,...,0.177,0.223,0.329,0.170,0.322,0.267,0.269,0.164,0.229,0.414
2018-05-01 04:00:00,0.235,0.199,0.107,0.009,0.047,0.131,0.153,0.255,0.139,0.169,...,0.129,0.171,0.301,0.168,0.351,0.255,0.170,0.132,0.198,0.318


In [95]:
total_size = len(df)
division_df = [df[_:_ + size]
                            for _ in range(0, total_size, size)]

In [102]:
mean_pattern = division_df[0].mean(axis=1)

In [118]:
def tss(mean_pattern, df):
    A = np.expand_dims(mean_pattern.values, axis=0)
    tss = 0 
    for col in df:
        B = np.expand_dims(df[col].values, axis=0)
        tss += euc(
                B,
                A
            )[0][0] ** 2
    return tss

In [119]:
tss(mean_pattern, division_df[0])

8.379768472222231

In [128]:
ci[0]

[array([[0.138, 0.13 , 0.136],
        [0.781, 0.671, 0.698],
        [0.632, 0.531, 0.388],
        [0.384, 0.349, 0.348],
        [0.198, 0.31 , 0.49 ],
        [0.269, 0.248, 0.226],
        [0.533, 0.279, 0.253]]),
 array([ 5., 27., 19., 14., 12.,  9., 13.])]

In [140]:
tdKMeans.tss_list

array([ 8.37976847,  4.54243562, 13.65746651, 10.3243494 ,  8.9892164 ,
       12.45286563, 28.39744917, 15.3067754 ])

In [155]:
tdKMeans.init_setting()
hc, ci = tdKMeans.run()

hc.head()

4.226452999999999 8.379768472222231


TypeError: cannot unpack non-iterable NoneType object

In [72]:
test = pd.DataFrame(df.values.reshape(-1, 3, 108).sum(axis=1),
                    columns=df.columns)
test.head()

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
0,1.002,0.521,1.017,0.028,0.217,0.420,0.522,0.757,0.714,0.507,...,0.862,0.669,0.901,1.076,1.257,1.171,0.575,0.479,0.690,1.018
1,0.723,0.587,0.708,0.027,0.153,0.419,0.511,0.802,0.452,0.489,...,0.527,0.632,0.938,0.534,1.029,0.775,0.660,0.642,0.655,1.061
2,0.541,0.375,0.392,0.027,0.111,0.541,0.924,0.658,0.575,0.780,...,0.335,1.807,1.597,1.116,1.275,1.343,1.015,0.790,1.324,1.249
3,0.540,0.485,0.194,0.028,0.385,0.794,0.905,0.839,1.677,0.446,...,0.459,1.863,0.925,1.259,1.319,1.455,0.409,0.935,0.835,1.381
4,0.597,0.380,0.272,0.027,0.283,0.637,0.899,0.775,1.338,0.493,...,1.033,1.322,1.084,1.172,1.724,0.899,0.394,0.747,0.702,1.772


In [91]:
def calc_contribution(hc, ci, df):
    _hc = hc.copy()
    _test = list()

    for col in _hc:
        _hc_info = _hc[col].copy()
        _hc_pattern = df[col].copy()

        contributions = list()
        for division_round, _ in enumerate(_hc_info):
            conts = ci[division_round][1]
            cont = conts[int(_)]
            _c_pattern = ci[division_round][0][int(_)]
            _h_pattern = _hc_pattern[division_round]

            if _c_pattern.sum() <= _h_pattern.sum():
                upper_conts = conts[conts > cont]
                if len(upper_conts) == 0:
                    contributions.append(
                        cont
                    )
                else:
                    upper_cont = upper_conts[upper_conts.argsort()][0]
                    
                    _conts = np.arange(upper_cont, cont)
                    _conts_max = len(_conts)
                
                    _percentage = (_h_pattern.sum() - _c_pattern.sum()) / _c_pattern.sum()
                    _new_cont_idx = round(_conts_max * _percentage)

                    if _new_cont_idx == _conts_max:
                        contributions.append(
                            cont
                        )
                    else:
                        contributions.append(
                            _conts[_new_cont_idx]
                        )
            else:
                lower_conts = conts[conts < cont]
                if len(lower_conts) == 0:
                    contributions.append(
                        cont
                    )
                else:
                    lower_cont = lower_conts[lower_conts.argsort()][::-1][0]
                    _conts = np.arange(lower_cont, cont)
                    _conts_max = len(_conts)
                    _percentage = _h_pattern.sum() / _c_pattern.sum()

                    _new_cont_idx = round(_conts_max * _percentage)
                    if _new_cont_idx == _conts_max:
                        contributions.append(
                            cont
                        )
                    else:
                        contributions.append(
                            _conts[_new_cont_idx]
                        )

        _test.append(contributions.copy())

    contribution_df = pd.DataFrame(_test).T
    contribution_df.columns = hc.columns

    return contribution_df.round()


In [92]:
contribution_df = calc_contribution(hc, ci, test)
contribution_df

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
0,15.0,7.0,15.0,3.0,3.0,6.0,7.0,11.0,11.0,7.0,...,15.0,11.0,15.0,15.0,20.0,19.0,7.0,7.0,11.0,15.0
1,12.0,12.0,13.0,3.0,3.0,8.0,9.0,17.0,9.0,9.0,...,9.0,13.0,17.0,9.0,17.0,17.0,12.0,13.0,12.0,20.0
2,7.0,3.0,3.0,3.0,3.0,7.0,11.0,8.0,7.0,8.0,...,3.0,17.0,19.0,14.0,14.0,14.0,14.0,8.0,14.0,14.0
3,7.0,7.0,3.0,3.0,6.0,11.0,12.0,12.0,20.0,6.0,...,6.0,25.0,12.0,17.0,17.0,17.0,6.0,12.0,12.0,17.0
4,8.0,4.0,4.0,4.0,4.0,8.0,9.0,9.0,18.0,4.0,...,14.0,14.0,13.0,14.0,18.0,9.0,4.0,9.0,9.0,18.0
5,8.0,3.0,3.0,3.0,3.0,8.0,13.0,8.0,13.0,21.0,...,13.0,25.0,13.0,21.0,21.0,13.0,13.0,8.0,21.0,21.0
6,4.0,4.0,4.0,4.0,4.0,7.0,24.0,11.0,15.0,11.0,...,11.0,11.0,17.0,15.0,17.0,8.0,11.0,8.0,8.0,11.0
7,11.0,5.0,5.0,5.0,5.0,5.0,11.0,16.0,8.0,11.0,...,16.0,11.0,21.0,11.0,27.0,16.0,11.0,8.0,11.0,16.0


In [93]:
# 실제(합산치 상에서의) 최소, 최대 사용량 가구
cols = df.columns

sum_df = (df.sum() * 1000).round() / 1000
min_h = cols[sum_df.argmin()]
max_h = cols[sum_df.argmax()]
print("실제 (합산치 상에서의) 최소, 최대 사용량 가구")
print("- 최소: {} ({}kwh) / 최대: {} ({}kwh)\n".format(
    min_h,
    sum_df[min_h],
    max_h,
    sum_df[max_h]
))

mean_cont_df = contribution_df.mean().round().astype("int")
_min_h = cols[mean_cont_df.argmin()]
_max_h = cols[mean_cont_df.argmax()]
print("기여도 최소, 최대 사용량 가구")
print("- 최소: {} ({}kwh) (기여도: {}%) / 최대: {} ({}kwh) (기여도: {}%)\n".format(
    _min_h,
    sum_df[min_h],
    mean_cont_df[_min_h],
    _max_h,
    sum_df[max_h],
    mean_cont_df[_max_h]
))
print("- 최소사용량 가구 기여도 , 최대사용량 가구 기여도 비교 {}배".format(round(mean_cont_df[_max_h] / mean_cont_df[_min_h])))

실제 (합산치 상에서의) 최소, 최대 사용량 가구
- 최소: 아파트1-104-1208 (0.219kwh) / 최대: 아파트1-103-703 (15.63kwh)

기여도 최소, 최대 사용량 가구
- 최소: 아파트1-104-1208 (0.219kwh) (기여도: 4%) / 최대: 아파트1-103-703 (15.63kwh) (기여도: 24%)

- 최소사용량 가구 기여도 , 최대사용량 가구 기여도 비교 6배
